# We want to create a synthetic time series data set

> Based on the synthetic data generation notebook of the ```afa``` repository by Henrik von Kleist.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pathlib
relative_path_to_afa = '../../afa_github/afa'
#sys.path.insert(0, os.path.abspath(relative_path_to_afa))
sys.path.insert(1, os.path.join(sys.path[0], relative_path_to_afa))
print('CWD:', pathlib.Path(pathlib.Path.cwd()))

import sklearn

# data generation
from afa_datasets.synthetic_data.data_generation_ts import generate_data_ts

# loading/ saving
from data_modelling.clairvoyance_static.datasets.data_loader import CSVLoader, save_dataset
from data_modelling.clairvoyance_static.utils.model_utils import PipelineComposer

CWD: /home2/joshua.wendland/Documents/sepsis/toy_dataset


2022-11-28 12:47:56.559341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-28 12:47:56.559367: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# define directories
data_dir = "dataset/"
dataset_name = "synthetic_1"

# Datageneration

Use sklearn's make-classification function

In [26]:
# df_train, df_test = generate_data_ts(data_dir = data_dir + dataset_name + "/", 
#                                          dataset_name = dataset_name,
#                                          n_features_static  = 4 ,
#                                          n_features_ts  = 10 ,
#                                          n_redundant = 0 ,
#                                          n_informative = 2,
#                                          n_datapts   = 200000,
#                                          n_classes   = 2,
#                                          random_state  = 47,
#                                          test_size   = 0.5)

data_train_ts_df, data_train_static_df, data_test_ts_df, data_test_static_df = generate_data_ts(data_dir = "",
                                        dataset_name = "synthetic_ts_1",
                                        n_features_static  = 2  ,
                                        n_features_ts      = 4  , 
                                        n_timepts   = 50,
                                        n_datapts   = 100, 
                                        test_size = 0.1,
                                        table_style = 'wide',
                                        n_classes   = 2)

# Load the Data

+ First import necessary stuff.
+ Look at simple implementations on how to handle the data
+ Then use ```Dataset``` class from Pytorch to handle the data.

In [1]:
import pandas as pd

Load the data and show the beginning.

In [2]:
path = '/home2/joshua.wendland/Documents/sepsis/toy_dataset/synthetic_ts_1/synthetic_ts_test_data_eav.csv.gz'

df = pd.read_csv(path, compression=None)
df = df.sort_values(by=['id', 'time'], ascending=True, ignore_index=True)  # time was not sorted

print(df.head)


<bound method NDFrame.head of         id  time    X_ts_0    X_ts_1    X_ts_2    X_ts_3      Y_ts
0    id_90     0  0.627422 -0.478959 -0.892121 -0.318527  1.083295
1    id_90     1 -0.448220  0.019303 -0.127440 -0.157412  1.093168
2    id_90     2  0.637957 -0.386629 -0.416992  0.772078  1.026738
3    id_90     3  0.114574 -0.700368  0.415711 -0.584559  0.891013
4    id_90     4  0.318873  0.112828  0.575051  0.149870  0.813476
..     ...   ...       ...       ...       ...       ...       ...
495  id_99    45  0.033853  0.889838  0.867147 -0.226947  0.981591
496  id_99    46 -0.883889  1.506605  0.732750  0.466535  0.846787
497  id_99    47  1.267585 -0.623064  0.034502 -0.162447  0.901342
498  id_99    48 -1.544032  0.310212  1.469414  0.941757  0.912135
499  id_99    49  1.052585 -0.451113  1.395424 -0.229020  0.822956

[500 rows x 7 columns]>


Group the data according to each ```id``` of the time series. 
This is easily done by using the ```groupby()``` method on the DataFrame. This transforms the data into the structure of:

| Group Name (or ID) | Pandas Series   |  
| ------------------ | --------------- |
| id_0              | Series of id_0 |  
| id_1              | Series of id_1 |  
| ...              | ... |  

To receive a certain Series ID ```i``` we just need to call ```df_grouped[i, 0]```.
To receive the corresponding Series data we call ```df_grouped[i, 1]```.

In [3]:
# Group by 'id'
df_grouped = pd.DataFrame(df.groupby('id'))
# Get rid of the 'id'-column in each group
for x in df_grouped.iloc[:,1]:
    x.drop(columns=['id'], inplace=True)
    
print(df_grouped.iloc[-1,1].head())
print(df_grouped.iloc[-1,1].tail())

     time    X_ts_0    X_ts_1    X_ts_2    X_ts_3      Y_ts
450     0  2.715997  0.325934 -0.486907 -2.543788  1.054109
451     1  1.526065  1.121756 -0.514032 -0.396498  0.764149
452     2 -0.505807  0.863921  0.798771 -0.444982  1.008917
453     3  0.788786  0.825285  0.303744 -1.426538  0.986686
454     4  0.219992 -0.188746 -0.861897 -0.611712  0.816993
     time    X_ts_0    X_ts_1    X_ts_2    X_ts_3      Y_ts
495    45  0.033853  0.889838  0.867147 -0.226947  0.981591
496    46 -0.883889  1.506605  0.732750  0.466535  0.846787
497    47  1.267585 -0.623064  0.034502 -0.162447  0.901342
498    48 -1.544032  0.310212  1.469414  0.941757  0.912135
499    49  1.052585 -0.451113  1.395424 -0.229020  0.822956


In [11]:
X = df_grouped.drop(columns=0).to_numpy(copy=True)


X = df_grouped.drop(columns=0)



(10, 1)


array([[    time    X_ts_0    X_ts_1    X_ts_2    X_ts_3      Y_ts
        0      0  0.627422 -0.478959 -0.892121 -0.318527  1.083295
        1      1 -0.448220  0.019303 -0.127440 -0.157412  1.093168
        2      2  0.637957 -0.386629 -0.416992  0.772078  1.026738
        3      3  0.114574 -0.700368  0.415711 -0.584559  0.891013
        4      4  0.318873  0.112828  0.575051  0.149870  0.813476
        5      5 -0.558442  2.029959  0.012359 -1.908553  0.898056
        6      6  0.939696  0.830056 -0.280783 -0.986594  0.935163
        7      7 -0.797461 -1.047239 -1.473412 -0.771835  0.762820
        8      8  1.885109 -0.629077 -1.387293 -0.917362  0.842649
        9      9 -1.015954  0.494451 -0.388469 -0.276064  1.019764
        10    10  0.468969 -0.523529 -0.680064  0.181355  0.972393
        11    11 -1.567821 -1.188615 -0.513788 -0.120319  0.855685
        12    12  0.788097  0.687057 -1.804858 -0.250144  1.185819
        13    13 -0.467239  1.484642 -0.442640  0.049348  0.89

## Using Pytorch class ```Dataset``` and ```Dataloader```

+ First import necessary modules.
+ Implement the ```Dataset``` class.

In [5]:
import torch
from torch.utils.data import Dataset
import pandas as pd

In [10]:
class ToyDataset(Dataset):

    def __init__(self, path, transform=None):
        df = pd.read_csv(path, compression=None)
        df = df.sort_values(by=['id', 'time'], ascending=True, ignore_index=True)  # time was not sorted
        # Number of samples
        self.n_samples = df['id'].unique()
        # Group by ID
        df_grouped = pd.DataFrame(df.groupby('id'))
        # Get rid of the 'id'-column in each group
        for x in df_grouped.iloc[:,1]:
            x.drop(columns=['id'], inplace=True)

        X = df_grouped.drop(columns=0).to_numpy(copy=True)

    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return self.n_samples

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.